In [1]:
import os
import sys
import time

project_root = os.path.abspath(os.path.join(os.path.dirname(''), '..'))
sys.path.insert(0, project_root)

print(f"Thư mục gốc dự án được thêm vào sys.path: {project_root}")
print(f"Current sys.path: {sys.path}")

from dotenv import load_dotenv
dotenv_path = os.path.join(project_root, 'backend', '.env')
load_dotenv(dotenv_path=dotenv_path) 
print(f"Đã cố gắng tải biến môi trường từ: {dotenv_path}")

from backend.utils.pinecone_utils import query_pinecone_index, initialize_pinecone_client, get_gemini_embedding
from demo.chatbot_app import detect_language, preprocess_query, get_gemini_llm_model, get_localized_error_message

PINECONE_INDEX_NAME = "apec2027-chatbot"
TOP_K_CHUNKS = 3
LLM_GENERATION_MODEL = 'gemini-2.0-flash'

try:
    initialize_pinecone_client()
    print("Đã sẵn sàng để kiểm tra quy trình RAG.")
except Exception as e:
    print(f"Lỗi khởi tạo Pinecone client: {e}")
    print("Vui lòng kiểm tra API Key và Environment của Pinecone trong .env.")

Thư mục gốc dự án được thêm vào sys.path: c:\Users\nguye\OneDrive\Desktop\Hekate\Case_01\apec2027_chatbot
Current sys.path: ['c:\\Users\\nguye\\OneDrive\\Desktop\\Hekate\\Case_01\\apec2027_chatbot', 'C:\\Users\\nguye\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip', 'C:\\Users\\nguye\\AppData\\Local\\Programs\\Python\\Python310\\DLLs', 'C:\\Users\\nguye\\AppData\\Local\\Programs\\Python\\Python310\\lib', 'C:\\Users\\nguye\\AppData\\Local\\Programs\\Python\\Python310', 'c:\\Users\\nguye\\OneDrive\\Desktop\\Hekate\\Case_01\\apec2027_chatbot\\venv', '', 'c:\\Users\\nguye\\OneDrive\\Desktop\\Hekate\\Case_01\\apec2027_chatbot\\venv\\lib\\site-packages', 'c:\\Users\\nguye\\OneDrive\\Desktop\\Hekate\\Case_01\\apec2027_chatbot\\venv\\lib\\site-packages\\win32', 'c:\\Users\\nguye\\OneDrive\\Desktop\\Hekate\\Case_01\\apec2027_chatbot\\venv\\lib\\site-packages\\win32\\lib', 'c:\\Users\\nguye\\OneDrive\\Desktop\\Hekate\\Case_01\\apec2027_chatbot\\venv\\lib\\site-packages\\Pythonwin']
Đ

c:\Users\nguye\OneDrive\Desktop\Hekate\Case_01\apec2027_chatbot\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Pinecone client initialized.
Đã sẵn sàng để kiểm tra quy trình RAG.


In [ ]:
from backend.utils.pinecone_utils import pc as pinecone_client_instance

if pinecone_client_instance:
    try:
        index_stats = pinecone_client_instance.describe_index_stats(PINECONE_INDEX_NAME)
        vector_count = index_stats.total_vector_count
        print(f"Tổng số vector đã được embedding trong index '{PINECONE_INDEX_NAME}': {vector_count}")
        if vector_count == 0:
            print("CẢNH BÁO: Index này hiện không có vector nào. Vui lòng chạy load_data_to_pinecone.py để đẩy dữ liệu lên.")
    except Exception as e:
        print(f"Lỗi khi kiểm tra số lượng vector trong Pinecone: {e}")
        print("Đảm bảo tên index và kết nối Pinecone chính xác.")
else:
    print("Pinecone client chưa được khởi tạo. Vui lòng chạy Cell 1 trước và kiểm tra lỗi.")

In [2]:
query_input = "APEC là gì?" # Thay đổi query của bạn tại đây

print(f"Truy vấn gốc của bạn: '{query_input}'")

start_time_lang_detect = time.time()
original_lang_code = detect_language(query_input)
end_time_lang_detect = time.time()
print(f"Ngôn ngữ gốc được phát hiện: {original_lang_code} (Thời gian: {end_time_lang_detect - start_time_lang_detect:.2f}s)")

start_time_preprocess = time.time()
processed_query_for_pinecone = preprocess_query(query_input, original_lang_code)
end_time_preprocess = time.time()
print(f"Query đã được xử lý (Fixed & Translated to English): '{processed_query_for_pinecone}' (Thời gian: {end_time_preprocess - start_time_preprocess:.2f}s)")

if not processed_query_for_pinecone:
    print(f"Lỗi: Query không được xử lý thành công. Kiểm tra lại API Key Gemini và kết nối mạng.")
else:
    start_time_embed_query = time.time()
    query_embedding = get_gemini_embedding(processed_query_for_pinecone, task_type="RETRIEVAL_QUERY")
    end_time_embed_query = time.time()

    if query_embedding:
        print(f"Query đã xử lý được Embedding thành công vào không gian vector.")
        print(f"Kích thước vector embedding: {len(query_embedding)} (Thời gian: {end_time_embed_query - start_time_embed_query:.2f}s)")
        print(f"Vài phần tử đầu tiên của vector: {query_embedding[:5]}...")
    else:
        print(f"Lỗi: Không thể embedding query đã xử lý. Vui lòng kiểm tra API Key Gemini và kết nối mạng.")

last_processed_query = processed_query_for_pinecone
last_original_lang_code = original_lang_code
print("\nHoàn tất kiểm tra xử lý query và embedding.")

Truy vấn gốc của bạn: 'APEC là gì?'
Ngôn ngữ gốc được phát hiện: vi (Thời gian: 3.23s)
Original Query: 'APEC là gì?'
Processed Query (Fixed & Translated to English): 'What is APEC?'
Query đã được xử lý (Fixed & Translated to English): 'What is APEC?' (Thời gian: 2.54s)
Query đã xử lý được Embedding thành công vào không gian vector.
Kích thước vector embedding: 768 (Thời gian: 1.41s)
Vài phần tử đầu tiên của vector: [0.013040245, 0.010783826, -0.009816597, 0.040476196, -0.0063064825]...

Hoàn tất kiểm tra xử lý query và embedding.


In [3]:
if not last_processed_query:
    print("Không có query đã xử lý để truy vấn. Vui lòng chạy Cell 3 trước.")
else:
    print(f"Đang truy vấn Pinecone với query đã xử lý: '{last_processed_query}'")
    start_time_retrieval = time.time()
    
    try:
        retrieved_chunks = query_pinecone_index(PINECONE_INDEX_NAME, last_processed_query, top_k=TOP_K_CHUNKS)
        end_time_retrieval = time.time()
        print(f"Thời gian xử lý truy xuất: {end_time_retrieval - start_time_retrieval:.2f}s")

        if retrieved_chunks:
            print(f"\nĐã truy xuất thành công {len(retrieved_chunks)} chunk liên quan (TOP {TOP_K_CHUNKS}):")
            for i, chunk in enumerate(retrieved_chunks):
                print(f"\n--- Chunk {i+1} (ID: {chunk['id']}) ---")
                print(f"Score: {chunk['score']:.4f}")
                print(f"Content (first 500 chars): {chunk['content'][:500]}...")
                print(f"Metadata: {chunk['metadata']}")
            print("--------------------------------------------------")
        else:
            print("Không tìm thấy chunk liên quan nào. Có thể dữ liệu không chứa thông tin hoặc score quá thấp.")

    except Exception as e:
        print(f"Lỗi khi truy vấn Pinecone: {e}")
        print("Vui lòng kiểm tra lại cấu hình Pinecone và API Key.")

last_retrieved_chunks = retrieved_chunks if 'retrieved_chunks' in locals() else []
print("\nHoàn tất kiểm tra truy xuất.")

Đang truy vấn Pinecone với query đã xử lý: 'What is APEC?'
Connected to index 'apec2027-chatbot'.
Thời gian xử lý truy xuất: 3.75s

Đã truy xuất thành công 3 chunk liên quan (TOP 3):

--- Chunk 1 (ID: chunk_0) ---
Score: 0.8127
Content (first 500 chars): The Asia-Pacific Economic Cooperation (APEC) is a regional economic forum established in 1989 to leverage the growing interdependence of the Asia-Pacific. APEC's 21 members aim to create greater prosperity for the people of the region by promoting balanced, inclusive, sustainable, innovative and secure growth and by accelerating regional economic integration.APEC ensures that goods, services, investment, and people move smoothly across borders. Members facilitate trade through streamlining custo...
Metadata: {'category': 'About APEC 2025 KOREA', 'original_text': "The Asia-Pacific Economic Cooperation (APEC) is a regional economic forum established in 1989 to leverage the growing interdependence of the Asia-Pacific. APEC's 21 members ai

In [4]:
if not last_retrieved_chunks or not last_processed_query:
    print("Không có chunk được truy xuất hoặc query đã xử lý. Vui lòng chạy các Cell trước.")
else:
    print(f"Đang tạo câu trả lời LLM cho query: '{last_processed_query}'")

    context_texts = [chunk["content"] for chunk in last_retrieved_chunks]
    context_texts = [text for text in context_texts if text != 'N/A' and text.strip()] 
    context_str = "\n\n---\n\n".join(context_texts)

    if not context_str.strip():
        print(f"Lỗi: Không thể tạo ngữ cảnh hợp lệ từ các thông tin tìm được. ({get_localized_error_message(last_original_lang_code, 'context_building_error')})")
    else:
        model = get_gemini_llm_model(model_name=LLM_GENERATION_MODEL)
        
        prompt = f"""Bạn là một trợ lý thông minh chuyên về thông tin APEC 2025.
        
        Bạn phải trả lời câu hỏi của người dùng một cách chính xác và đầy đủ nhất DỰA TRÊN ngữ cảnh được cung cấp dưới đây.
        Nếu ngữ cảnh không chứa đủ thông tin để trả lời, hãy nói rằng bạn không biết thông tin đó.
        Không bịa đặt thông tin.

        Trả lời bằng ngôn ngữ của CÂU HỎI GỐC CỦA NGƯỜI DÙNG.
        Ví dụ: Nếu CÂU HỎI GỐC CỦA NGƯỜI DÙNG là tiếng Việt, trả lời bằng tiếng Việt. Nếu là tiếng Anh, trả lời bằng tiếng Anh. Nếu là tiếng Hàn, trả lời bằng tiếng Hàn.
        
        QUAN TRỌNG: LUÔN GIỮ NGUYÊN TÊN ĐỊA ĐIỂM, TÊN SỰ KIỆN, TÊN TỔ CHỨC, THỜI GIAN, NGÀY THÁNG, SỐ ĐIỆN THOẠI, WEBSITE, và CÁC THUẬT NGỮ CHUYÊN MÔN bằng tiếng Anh trong câu trả lời.
        Ví dụ: "Hội nghị APEC Economic Leaders’ Meeting sẽ diễn ra tại Gyeongju."
        
        [NGỮ CẢNH]
        {context_str}

        [CÂU HỎI GỐC CỦA NGƯỜI DÙNG]
        {query_input}

        [TRẢ LỜI]
        """
        
        print("--- LLM Prompt (Final Generation) ---")
        print(prompt)
        print("-------------------------------------")

        start_time_llm_gen = time.time()
        try:
            response = model.generate_content(prompt)
            end_time_llm_gen = time.time()
            final_answer = response.text if response.candidates and response.candidates[0].content.parts else ""
            
            if not final_answer:
                print(f"Lỗi: LLM response rỗng hoặc bị chặn. ({get_localized_error_message(last_original_lang_code, 'llm_generation_error')})")
            else:
                print(f"Thời gian tạo câu trả lời LLM: {end_time_llm_gen - start_time_llm_gen:.2f}s")
                print("\n--- CÂU TRẢ LỜI TỪ LLM ---")
                print(final_answer)
                print("--------------------------")
        except Exception as e:
            print(f"Lỗi khi tạo câu trả lời với LLM: {e} ({get_localized_error_message(last_original_lang_code, 'llm_generation_error')})")

print("\nHoàn tất kiểm tra tạo câu trả lời LLM.")

Đang tạo câu trả lời LLM cho query: 'What is APEC?'
--- LLM Prompt (Final Generation) ---
Bạn là một trợ lý thông minh chuyên về thông tin APEC 2025.
        
        Bạn phải trả lời câu hỏi của người dùng một cách chính xác và đầy đủ nhất DỰA TRÊN ngữ cảnh được cung cấp dưới đây.
        Nếu ngữ cảnh không chứa đủ thông tin để trả lời, hãy nói rằng bạn không biết thông tin đó.
        Không bịa đặt thông tin.

        Trả lời bằng ngôn ngữ của CÂU HỎI GỐC CỦA NGƯỜI DÙNG.
        Ví dụ: Nếu CÂU HỎI GỐC CỦA NGƯỜI DÙNG là tiếng Việt, trả lời bằng tiếng Việt. Nếu là tiếng Anh, trả lời bằng tiếng Anh. Nếu là tiếng Hàn, trả lời bằng tiếng Hàn.
        
        QUAN TRỌNG: LUÔN GIỮ NGUYÊN TÊN ĐỊA ĐIỂM, TÊN SỰ KIỆN, TÊN TỔ CHỨC, THỜI GIAN, NGÀY THÁNG, SỐ ĐIỆN THOẠI, WEBSITE, và CÁC THUẬT NGỮ CHUYÊN MÔN bằng tiếng Anh trong câu trả lời.
        Ví dụ: "Hội nghị APEC Economic Leaders’ Meeting sẽ diễn ra tại Gyeongju."
        
        [NGỮ CẢNH]
        The Asia-Pacific Economic Cooperation (A